In [28]:
import pandas as pd
import numpy as np
import unicodedata
from bs4 import BeautifulSoup
from pytorch_transformers import BertTokenizer 

import re

df = pd.read_csv('IMDB_Dataset.csv', encoding='utf-8')
# df = df.loc[:10]
# 시험삼아 해보세용..

In [29]:
# Sentiment 1, 0으로 바꾸기
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

In [30]:
# 문장 단위로 쪼개기
df_reviews = df.review.str.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', expand=True)
df_reviews = df_reviews.stack().reset_index(level=1, drop=True).to_frame('review_single')
df_pre = df[["sentiment"]].merge(df_reviews, left_index=True, right_index=True, how='left')
df_reviews = df_pre.values

In [31]:
counter = 0
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [32]:
# 전처리
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(raw_review: str) -> str:
    # 1. unicode to ASCII
    raw_review = unicodeToAscii(raw_review)
    
    # 2. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    
    return review_text

def preprocess(review: str, sentiment: int, total: int, show_progress: bool = True) -> list:
    if show_progress:
        global counter
        counter += 1
        print('Processing... %6i/%6i'% (counter, total), end='\r')
    # 1. Clean text
    review = '[CLS] ' + clean_review(review)
        
    # 2. BERT
    tokens = tokenizer.tokenize(review)
    all_tokens = []
    for i in range(min(len(tokens),512)):
        token_ids = tokenizer.convert_tokens_to_ids(tokens[:i+1])
        all_tokens.append([token_ids, sentiment])
    return all_tokens

In [33]:
df_reviews = np.array(list(map(lambda x: preprocess(x[1], x[0], len(df_reviews)), df_reviews)))

/usr/local/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


/usr/local/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [35]:
np.savez('./IMDB_pre_Dataset', df_reviews)